In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Code Example: Create a feature distribution graph by styles over decades - songs that contain a bridge section

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import numpy as np
sns.set(color_codes=True)


def sort_decades(dec_list, name):
    dframe = pd.DataFrame(dec_list)
    a = pd.DataFrame(index=['1970s', '1980s', '1990s', '2000s', '2010s'])
    a[0] = 0
    b = dframe[0].value_counts()
    a.update(b)
    dframe = pd.DataFrame()
    dframe['count'] = a[0]
    dframe['style'] = name
    dframe['decade'] = dframe.index
    return dframe


# main
df = pd.read_csv('/kaggle/input/1001-israeli-pop-dataset/1001dataset.csv')
cols = ['DECADES', 'IS.MIZ', 'IS.ROCK']
# filter_cols = ['R.VERSE', 'R.CHORUS', 'R3']     # rhythmic pattern columns
# filter_cols = list(df.columns[38:83])              # harmonic analysis columns
# filter_cols = [col for col in df if col.startswith('INST')]       # timbres columns
# cols += filter_cols
filter_cols = list(df.columns[19:38])              # form-related columns
# filter_cols = list(df.columns[101:])              # all lyrics-related columns (topics, lyrics, pronouns, tags)
# filter_cols = [col for col in df if col.startswith('TOPIC')]       # lyrical topics columns
# cols += filter_cols
# filter_cols = [col for col in df if col.startswith('LYRIC') and not col.endswith('IST')]   # lyrics columns
# filter_cols = [col for col in df if col.startswith('TAG')]      # lyrical tags columns
# cols += ['PRONOUNS']
# filter_cols = ['COMPOSER']
# minor = ['IS.MINOR']
cols += filter_cols
# cols += ['vc', 'cycles']
miz = []
notmiz = []
rock = []

title = 'distribution of bridge formal section'
for index, row in df[cols].iterrows():
    if ('bridge') in str(row):
        if row['IS.MIZ']:
            miz.append(row['DECADES'])
        else:
            notmiz.append(row['DECADES'])
        if row['IS.ROCK']:
            rock.append(row['DECADES'])

miz = sort_decades(miz, 'MIZ')
# notmiz = sort_decades(notmiz, 'notMIZ')
rock = sort_decades(rock, 'ROCK')
# chart = pd.concat([miz, notmiz])
chart = pd.concat([miz, rock])
chart.reset_index(drop=True, inplace=True)
all_sums = pd.read_pickle('/kaggle/input/1001-israeli-pop-dataset/mizrock_dist_by_decade.pkl')
# all_sums = pd.read_pickle('/kaggle/input/1001-israeli-pop-dataset/miznotmiz_dist_by_decade.pkl')    # alternate distribution
all_sums = all_sums.reset_index()
chart['frequency'] = chart['count'] / all_sums[0]
miz = chart.iloc[:5, :]
rock = chart.iloc[5:, :]

plt.figure('dist', figsize=(6, 4))
num_decades = 5
ax = sns.barplot(x='decade', y='frequency', hue='style', data=chart)
sns.regplot(x=np.arange(0, 5), y='frequency', data=miz, ax=ax, scatter=False, label='miz', color='dodgerblue')
sns.regplot(x=np.arange(0, 5), y='frequency', data=rock, ax=ax, scatter=False, label='rock', color='orange',
            line_kws={'ls':'--'})
plt.ylim(0, 1)
plt.xlim(-0.5, 4.5)
hatches = itertools.cycle(['', '//'])
for i, bar in enumerate(ax.patches):
    if i % num_decades == 0:
        hatch = next(hatches)
    bar.set_hatch(hatch)
plt.legend(framealpha=0.9, fancybox=True)   # bbox_to_anchor=(0.95, 0.1), loc='right')
plt.title(title)
plt.tight_layout()
plt.show()


In [ ]:
# Code Example: Create a feature distribution graph by styles over decades  - lyrical topics and tags about faith, Judaism or religion

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import numpy as np
sns.set(color_codes=True)


def sort_decades(dec_list, name):
    dframe = pd.DataFrame(dec_list)
    a = pd.DataFrame(index=['1970s', '1980s', '1990s', '2000s', '2010s'])
    a[0] = 0
    b = dframe[0].value_counts()
    a.update(b)
    dframe = pd.DataFrame()
    dframe['count'] = a[0]
    dframe['style'] = name
    dframe['decade'] = dframe.index
    return dframe


# main
df = pd.read_csv('/kaggle/input/1001-israeli-pop-dataset/1001dataset.csv')
cols = ['DECADES', 'IS.MIZ', 'IS.ROCK']
# filter_cols = ['R.VERSE', 'R.CHORUS', 'R3']     # rhythmic pattern columns
# filter_cols = list(df.columns[38:83])              # harmonic analysis columns
# filter_cols = [col for col in df if col.startswith('INST')]       # timbres columns
# cols += filter_cols
# filter_cols = list(df.columns[19:38])              # form-related columns
# filter_cols = list(df.columns[101:])              # all lyrics-related columns (topics, lyrics, pronouns, tags)
filter_cols = [col for col in df if col.startswith('TOPIC')]       # lyrical topics columns
cols += filter_cols
# filter_cols = [col for col in df if col.startswith('LYRIC') and not col.endswith('IST')]   # lyrics columns
filter_cols = [col for col in df if col.startswith('TAG')]      # lyrical tags columns
# cols += ['PRONOUNS']
# filter_cols = ['COMPOSER']
# minor = ['IS.MINOR']
cols += filter_cols
# cols += ['vc', 'cycles']
miz = []
notmiz = []
rock = []

title = 'songs about faith, Judaism and religion (topics or lyrical tags)'
for index, row in df[cols].iterrows():
    if ('אמונה' or 'יהדות' or 'אלוהים' or 'דת' or 'תפילה' or 'יהודי') in str(row):
        if row['IS.MIZ']:
            miz.append(row['DECADES'])
        else:
            notmiz.append(row['DECADES'])
        if row['IS.ROCK']:
            rock.append(row['DECADES'])

miz = sort_decades(miz, 'MIZ')
# notmiz = sort_decades(notmiz, 'notMIZ')
rock = sort_decades(rock, 'ROCK')
# chart = pd.concat([miz, notmiz])
chart = pd.concat([miz, rock])
chart.reset_index(drop=True, inplace=True)
all_sums = pd.read_pickle('/kaggle/input/1001-israeli-pop-dataset/mizrock_dist_by_decade.pkl')
# all_sums = pd.read_pickle('/kaggle/input/1001-israeli-pop-dataset/miznotmiz_dist_by_decade.pkl')    # alternate distribution
all_sums = all_sums.reset_index()
chart['frequency'] = chart['count'] / all_sums[0]
miz = chart.iloc[:5, :]
rock = chart.iloc[5:, :]

plt.figure('dist', figsize=(6, 4))
num_decades = 5
ax = sns.barplot(x='decade', y='frequency', hue='style', data=chart)
sns.regplot(x=np.arange(0, 5), y='frequency', data=miz, ax=ax, scatter=False, label='miz', color='dodgerblue')
sns.regplot(x=np.arange(0, 5), y='frequency', data=rock, ax=ax, scatter=False, label='rock', color='orange',
            line_kws={'ls':'--'})
plt.ylim(0, 1)
plt.xlim(-0.5, 4.5)
hatches = itertools.cycle(['', '//'])
for i, bar in enumerate(ax.patches):
    if i % num_decades == 0:
        hatch = next(hatches)
    bar.set_hatch(hatch)
plt.legend(framealpha=0.9, fancybox=True)   # bbox_to_anchor=(0.95, 0.1), loc='right')
plt.title(title)
plt.tight_layout()
plt.show()
